In [ ]:
import sqlite3

# Connect to the database using a properly formatted path.
connection = sqlite3.connect("../databases/takeaway.db")

cursor = connection.cursor()

B1: What is the price distribution of menu items?

In [38]:

# Fetch all results and store them in a variable.
results = cursor.fetchall()

# Print the first 5 fetched results.
print("Sample Data (First 5 Rows):")
#print(results[:10])

# Step 2: Calculate price distribution 
cursor.execute("""
    SELECT 
        CASE 
            WHEN price <= 10 THEN '0-10'
            WHEN price > 10 AND price <= 20 THEN '10-20'
            WHEN price > 20 AND price <= 30 THEN '20-30'
            ELSE '30+'
        END AS price_range,
        COUNT(*) AS item_count
    FROM menuItems
    GROUP BY price_range;
""")

# Fetch the price distribution results.
price_distribution = cursor.fetchall()

# Print the price distribution.
print("\nPrice Distribution:")
for range_, count in price_distribution:
    print(f"Price Range {range_}: {count} items")

# Close the connection when done.
connection.close()


Sample Data (First 5 Rows):

Price Distribution:
Price Range 0-10: 230390 items
Price Range 10-20: 149249 items
Price Range 20-30: 18073 items
Price Range 30+: 5301 items


O2: Which restaurants have the best price-to-rating ratio?

In [ ]:

# SQL query to calculate the best price-to-rating ratio and include address and city
query_price_to_rating_ratio = """
                                         -- Step 1: Calculate the average price for menu items grouped by primarySlug
    WITH AvgMenuPrices AS (
        SELECT 
            primarySlug,
            AVG(price) AS avg_price
        FROM 
            menuItems
        GROUP BY 
            primarySlug
    )
                                        -- Step 2: Join the average prices with restaurant details
    SELECT 
        restaurants.name AS restaurant_name,
        restaurants.address AS restaurant_address,
        restaurants.city AS restaurant_city,
        AvgMenuPrices.avg_price / restaurants.ratings AS price_to_rating_ratio
    FROM 
        restaurants
    JOIN 
        AvgMenuPrices ON restaurants.primarySlug = AvgMenuPrices.primarySlug
    WHERE 
        restaurants.ratings > 0         -- Exclude restaurants with zero or null ratings
    ORDER BY 
        price_to_rating_ratio DESC      -- Sort by the best ratio (highest first)
    LIMIT 10;                           -- Display top 5 restaurants
"""

# Execute the query
cursor.execute(query_price_to_rating_ratio)

# Fetch and display the results
results = cursor.fetchall()
print("Top Restaurants with Best Price-to-Rating Ratio:")
for row in results:
    restaurant_name, restaurant_address, restaurant_city, ratio = row
    #print(f"Restaurant: {restaurant_name}, Address: {restaurant_address}, City: {restaurant_city}, Price-to-Rating Ratio: {ratio:.2f}")
    print(f" {ratio:.2f}   {restaurant_name},  address: {restaurant_address}, {restaurant_city}")
# Close the connection
connection.close()



Top Restaurants with Best Price-to-Rating Ratio:
 9.13   Pizza Hut Delivery,  address: Hubert d'Ydewallestraat 32, Brugge
 8.74   Japans Restaurant Amatsu,  address: Hoogpoort 29, Gent
 8.26   Pizza Hut Delivery,  address: Torhoutsesteenweg 270, Brugge Sint-Andries
 8.20   Wok van Antwerpen,  address: Carnotstraat 123, Antwerpen
 7.85   Mère Liban,  address: Kustlaan 24, Knocke-Heist
 7.80   Drank Express,  address: Kortrijksesteenweg 77, Harelbeke
 7.58   Little Ethiopia,  address: Zirkstraat 8, Antwerpen
 7.45   New Restaurant Henri 1er,  address: Avenue de Messidor 181, Bruxelles
 6.96   D-Afro Flavours,  address: Naamsestraat 87, Leuven
 6.93   Mai Thai,  address: Suikerrui 26, Antwerpen


O5. How do ratings of similar restaurants differ across Uber Eats, Deliveroo, and Takeaway?

In [9]:
import sqlite3
import pandas as pd

#---------------------------------------------------------------------------------------

# Paths to the three databases
# Paths to the individual SQLite database files for Uber Eats, Deliveroo, and Takeaway
uber_db = "../databases/ubereats.db"
deliveroo_db = "../databases/deliveroo.db"
takeaway_db = "../databases/takeaway.db"

# Connect to each database
# Establishing connections to the databases
uber_connection = sqlite3.connect(uber_db)
deliveroo_connection = sqlite3.connect(deliveroo_db)
takeaway_connection = sqlite3.connect(takeaway_db)

#---------------------------------------------------------------------------------------

# Step 1: Extract restaurant ratings from Uber Eats
# Query to extract restaurant name and rating from the Uber Eats database
uber_query = """
    SELECT 
        slug AS restaurant_name,                  -- Selecting the column 'slug' and renaming it to 'restaurant_name'
        rating__rating_value AS uber_ratings      -- Selecting the column 'rating__rating_value' and renaming it to 'uber_ratings'
    FROM restaurants;                             -- Querying the 'restaurants' table in the Ubereats database
"""
uber_data = pd.read_sql_query(uber_query, uber_connection)

# Step 2: Extract restaurant ratings from Deliveroo
# Query to extract restaurant name and rating from the Deliveroo database
deliveroo_query = """
    SELECT 
        uname AS restaurant_name, 
        rating AS deliveroo_ratings 
    FROM restaurants; 
"""
deliveroo_data = pd.read_sql_query(deliveroo_query, deliveroo_connection)

# Step 3: Extract restaurant ratings from Takeaway
# Query to extract restaurant name and rating from the Takeaway database
takeaway_query = """
    SELECT 
        primarySlug AS restaurant_name,
        ratings AS takeaway_ratings 
    FROM restaurants; 
"""
takeaway_data = pd.read_sql_query(takeaway_query, takeaway_connection)

# Close individual database connections
# Closing connections to the individual databases after extracting the data
uber_connection.close()
deliveroo_connection.close()
takeaway_connection.close()

#---------------------------------------------------------------------------------------

# Step 4: Write data to an in-memory SQLite database for merging                                # (this part made by my cousin, ChatGPT)      
# Transferring the extracted data into an in-memory SQLite database for easier merging
# so does not need to change the name of columns on main databases
connection = sqlite3.connect(":memory:")                                                        # Creating an in-memory SQLite database
uber_data.to_sql("uber_restaurants", connection, if_exists="replace", index=False)              # Writing Uber Eats data to the 'uber_restaurants' table
deliveroo_data.to_sql("deliveroo_restaurants", connection, if_exists="replace", index=False)    # Writing Deliveroo data to the 'deliveroo_restaurants' table
takeaway_data.to_sql("takeaway_restaurants", connection, if_exists="replace", index=False)      # Writing Takeaway data to the 'takeaway_restaurants' table

#---------------------------------------------------------------------------------------

# Step 5: Merge the data
# SQL query to merge the data from the three datasets and calculate comparison metrics
merged_query = """

    SELECT 

        -- select the coumns and making alias for each columns
        uber_data.restaurant_name AS uber_restaurant_name,                     -- Ubereats restaurant name
        uber_data.uber_ratings,                                                -- Ubereats ratings

        deliveroo_data.restaurant_name AS deliveroo_restaurant_name,           -- Deliveroo restaurant name
        deliveroo_data.deliveroo_ratings,                                      -- Deliveroo ratings

        takeaway_data.restaurant_name AS takeaway_restaurant_name,             -- Takeaway restaurant name
        takeaway_data.takeaway_ratings,                                        -- Takeaway ratings

        -- -------------------------------------------------------------
        -- Calculate the average rating across the three platforms

        (uber_data.uber_ratings + 
            deliveroo_data.deliveroo_ratings + 
            takeaway_data.takeaway_ratings) / 3.0 AS average_rating,

        -- -------------------------------------------------------------
        -- Calculate the absolute difference between databases  (regardless of whether it is positive or negative)
        -- to identifying the extent of differences in how each platform scores

        ABS(uber_data.uber_ratings - deliveroo_data.deliveroo_ratings) AS uber_vs_deliveroo_diff,               --  difference between Ubereats and Deliveroo ratings                                                                        
        ABS(uber_data.uber_ratings - takeaway_data.takeaway_ratings) AS uber_vs_takeaway_diff,                  --  difference between Ubereats and Takeaway ratings                                                             
        ABS(deliveroo_data.deliveroo_ratings - takeaway_data.takeaway_ratings) AS deliveroo_vs_takeaway_diff    -- difference between Deliveroo and Takeaway ratings

    FROM
        uber_restaurants AS uber_data                                          -- Using the table containing Ubereats data

    INNER JOIN                                                                 -- (INNER JOIN) is only selects restaurants that are listed on all three platforms
        deliveroo_restaurants AS deliveroo_data                                -- Joining with the Deliveroo data
    ON 
        uber_data.restaurant_name = deliveroo_data.restaurant_name             -- (ON) is matching restaurants by name

    INNER JOIN
        takeaway_restaurants AS takeaway_data                                  -- Joining with the Takeaway data
    ON 
        uber_data.restaurant_name = takeaway_data.restaurant_name             -- Matching restaurants by name

    ORDER BY average_rating DESC
    LIMIT 5;

"""

# Step 6: Execute the merged query
# Executing the merge query to combine data from the three platforms and calculate metrics
merged_data = pd.read_sql_query(merged_query, connection)

#---------------------------------------------------------------------------------------

# Step 7: Display the results
# Printing the merged data with ratings comparison
print("Ratings Comparison Across Platforms:")
print(merged_data)

# Close the in-memory database connection
# Closing the in-memory SQLite database connection after processing
connection.close()


Ratings Comparison Across Platforms:
  uber_restaurant_name  uber_ratings deliveroo_restaurant_name  \
0                 cali           4.9                      cali   
1          tmore-enjoy           5.0               tmore-enjoy   
2       janya-thai-wok           5.0            janya-thai-wok   
3              hey-joo           4.8                   hey-joo   
4         collina-2-go           4.6              collina-2-go   

   deliveroo_ratings takeaway_restaurant_name  takeaway_ratings  \
0                4.6                     cali               5.0   
1                4.6              tmore-enjoy               4.6   
2                4.5           janya-thai-wok               4.3   
3                4.6                  hey-joo               4.4   
4                4.6             collina-2-go               4.6   

   average_rating  uber_vs_deliveroo_diff  uber_vs_takeaway_diff  \
0        4.833333                     0.3                    0.1   
1        4.733333          